In [1]:
%matplotlib inline
import pandas as pd
import matplotlib as plt
import pandas_profiling
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import pylab as pl
from sklearn.model_selection import train_test_split
plt.style.use('ggplot')

/Users/monnedepraetere/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/monnedepraetere/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
/Users/monnedepraetere/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [106]:
# read in data, create column that identifies the type of dataset
train = pd.read_csv('/Users/monnedepraetere/Google Drive/3. Coding/Titanic/train.csv')
train['Dataset'] = 'train'
test = pd.read_csv('/Users/monnedepraetere/Google Drive/3. Coding/Titanic/test.csv')
test['Dataset'] = 'test'
test['Survived'] = 0

# make sure the two dataframes have same column order, then concatenate
columns = list(train.columns)
columns.remove('Survived')
columns.append('Survived')
test = test.reindex(columns=columns)
train = train.reindex(columns=columns)
df = pd.concat([train,test]).set_index('PassengerId')

# add a column for age group
age_categories = ['Very Young', 'Young','Average','Old','Very Old']
df['Age_group'] = pd.qcut(df['Age'],5, age_categories)

# add a column for higher class
less_than_three = lambda x: 1 if x < 3 else 0
df['Higher_class'] = df['Pclass'].apply(less_than_three)

# create dummies for class, age and sex, merge into train dataframe and drop columns we don't need
dummy_class = pd.get_dummies(df['Pclass'],prefix='class').drop('class_3',1)
dummy_male = pd.get_dummies(df['Sex']).drop('female', 1)
df = df.join(dummy_male).join(dummy_class)

#larger_than_zero = lambda x: 1 if x > 0 else 0
#df['Has_children'] = df['SibSp'].apply(larger_than_zero)
#df['Has_parent'] = df['Parch'].apply(larger_than_zero)
#df['Part_of_family'] = df['Has_children'] + df['Has_parent']
#df['Part_of_family'] = df['Part_of_family'].apply(lambda x: 1 if x>0 else 0)

In [107]:
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Dataset,Survived,Age_group,Higher_class,male,class_1,class_2
PassengerId,,,,,,,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,train,0,Young,0,1,0,0
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,train,1,Old,1,0,1,0
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,train,1,Average,0,0,0,0
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,train,1,Old,1,0,1,0
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,train,0,Old,0,1,0,0


In [108]:
# split up the dataframe in a train and test dataframe
df = df[df['Dataset']=='train'] 
x = df
y = pd.DataFrame(df['Survived'])

# filter out the relevant columns
columns_to_keep = ['male','Higher_class']
x = x[columns_to_keep]

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size=.20)

In [109]:
# logistic regression
logit = sm.Logit(y_train,x_train)
result = logit.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.474205
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  712
Model:                          Logit   Df Residuals:                      710
Method:                           MLE   Df Model:                            1
Date:                Sun, 21 May 2017   Pseudo R-squ.:                  0.2840
Time:                        10:39:37   Log-Likelihood:                -337.63
converged:                       True   LL-Null:                       -471.54
                                        LLR p-value:                 3.400e-60
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
male            -2.3506      0.170    -13.829      0.000      -2.684      -2.017
Higher_class     1.6828      0.178      9.479      0.000       1.335       2.031
================================================================================
"""

In [110]:
# make predictions
y_test_model = result.predict(x_test).apply(lambda x: round(x,0)).astype(int)

In [111]:
y_test = pd.DataFrame(y_test)
y_test_model = pd.DataFrame(y_test_model)

In [112]:
y_test.columns = ['Survived_actual']
y_test_model.columns = ['Survived_modelled']

In [113]:
y_test_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 179 entries, 710 to 11
Data columns (total 1 columns):
Survived_modelled    179 non-null int64
dtypes: int64(1)
memory usage: 2.8 KB


In [114]:
results_comparison = pd.merge(y_test,y_test_model,left_index=True, right_index=True)
results_comparison['count']=1

In [115]:
pivot = pd.pivot_table(results_comparison,values=['count'],index='Survived_actual',
               columns='Survived_modelled',aggfunc='count') / results_comparison['count'].count()
success_rate = 100 * (pivot.iloc[0,0] + pivot.iloc[1,1])
print "This model using variables 'sex' and 'class' gives us an estimated success rate of {:.0f}%".format(success_rate)

This model using variables 'sex' and 'class' gives us an estimated success rate of 78%


In [117]:
pivot

count          
Survived_modelled         0         1
Survived_actual                      
0                  0.491620  0.094972
1                  0.122905  0.290503